# Init

In [13]:
import pandas as pd
import os
import glob
from datetime import datetime

In [14]:
#setting path to the env variable
folder = os.environ['CAPSTONE_DATA']
folder

'C:\\max\\Google Drive Max\\Max Shared\\Master Data Science\\2020-05 Capstone A\\msds_capstone\\data\\'

# ETL Tranasactions

## Merging original data

In [15]:
path = os.path.join(folder,'original_untidy/Transactions_v3/')
files =  glob.glob(path + "*.xlsx")
dataRaw = pd.DataFrame()
for i,f in enumerate(files):
    print(f)
    dataRaw = dataRaw.append(pd.read_excel(f,skiprows=8),ignore_index=True)

dataRaw=dataRaw.rename(columns={"Chain Master":"Chain Master ID"
                                ,"Unnamed: 2":"Chain Master"
                                ,"Customer":"Customer ID"
                                ,"Unnamed: 4": "Customer"
                                ,"Unnamed: 7":"Vendor ID"
                                ,"Unnamed: 12":"Product ID"
                                ,'Unnamed: 38':'Product Type ID'
                               })
                                
dataRaw.head()


C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\original_untidy/Transactions_v3\Bivin ELP 2013-2016.xlsx
C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\original_untidy/Transactions_v3\Bivin ELP 2017-2019.xlsx


,Month,Chain Master ID,Chain Master,Customer ID,Customer,Beverage Type,Vendor,Vendor ID,Brand,Label,...,Category,Product Type,Unnamed: 34,Qty Per Case,Alcohol Proof,STD. Cases,Dollar Sales,9L Cases,Dollar Sales per Case,Dollar Sales per 9L Case
0,Nov-2015,2577,JUANITOS,300001611,JUANITOS LIQUOR STORE INC,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,TAAKA,TAAKA VODKA 80,...,VODKA MID PRICE,VODKA,424,12,80.0,350.0,20412.00,465.50,58.320000,43.849624
1,Aug-2016,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,BOURBON PREM PRICE,WHISKEY,434,12,80.0,322.0,96522.72,428.26,299.760000,225.383459
2,May-2016,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,BOURBON PREM PRICE,WHISKEY,434,12,80.0,314.0,92706.24,417.62,295.242803,221.987070
3,Dec-2015,2577,JUANITOS,300001611,JUANITOS LIQUOR STORE INC,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,TAAKA,TAAKA VODKA 80,...,VODKA MID PRICE,VODKA,424,12,80.0,250.0,14580.00,332.50,58.320000,43.849624
4,Dec-2016,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,BOURBON PREM PRICE,WHISKEY,434,12,80.0,216.0,64748.16,287.28,299.760000,225.383459


## Transformation

In [16]:
# merge iwht STD Cases Table
# https://www.ttb.gov/distilled-spirits/conversion-tables 
cases = pd.DataFrame({"Size":['750 ML', '1.75 LIT / 1.5L', '100 ML', '375 ML', '1.0 LITER','200/187 ML', '50 ML','3 LITER OR LARGER']
                      ,"Size Liters":[.750,1.75 ,.1 ,.375 ,.1 ,.2 ,.05,3]
                      #,"Case Bottles":[12  ,6    ,60 ,24   ,12 ,48 ,120]
                      #,"Case Liters": [9   ,10.50,6  ,9    ,9  ,9.6,6]
                     })


data=pd.merge(dataRaw,cases,how='left',on="Size")

#fix Month
data['Month'] = pd.to_datetime(data['Month'],format="%b-%Y")

#calculate totals
# https://www.inchcalculator.com/convert/case-beer-to-liter-beer/
data['Bottles']=data['Qty Per Case'] * data['STD. Cases']
#data['Liters']=data['Size Liters'] *  data['Bottles'] # conversion is more complex see above link, we need to use the 9l cases unit 
#data['Dollar Sales per Liter']=data['Dollar Sales per 9L Case'] * 9.0  # conversion is more complex see above link, we need to use the 9l cases unit

#adding a 'transactions' column
data['Transactions']=1

#Removing nulls in dollar sales (it happens whnere STD Cases is 0 and there is a credit in Dollar Sales)
data.dropna(subset=['STD. Cases','Dollar Sales per Case','Dollar Sales per 9L Case'],inplace=True)

## Remove negatives!
print(data.shape)
cols=['STD. Cases','Dollar Sales','9L Cases','Dollar Sales per Case','Dollar Sales per 9L Case']
for c in cols: data=data.loc[data[c]>0]
print(data.shape)
print('check for negtives',[sum(data[c]<=0) for c in cols])

data.head()


(54360, 45)
(53552, 45)
check for negtives [0, 0, 0, 0, 0]


,Month,Chain Master ID,Chain Master,Customer ID,Customer,Beverage Type,Vendor,Vendor ID,Brand,Label,...,Qty Per Case,Alcohol Proof,STD. Cases,Dollar Sales,9L Cases,Dollar Sales per Case,Dollar Sales per 9L Case,Size Liters,Bottles,Transactions
0,2015-11-01,2577,JUANITOS,300001611,JUANITOS LIQUOR STORE INC,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,TAAKA,TAAKA VODKA 80,...,12,80.0,350.0,20412.00,465.50,58.320000,43.849624,0.1,4200.0,1
1,2016-08-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,12,80.0,322.0,96522.72,428.26,299.760000,225.383459,0.1,3864.0,1
2,2016-05-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,12,80.0,314.0,92706.24,417.62,295.242803,221.987070,0.1,3768.0,1
3,2015-12-01,2577,JUANITOS,300001611,JUANITOS LIQUOR STORE INC,SPIRITS,SAZERAC/GEMINI SPIRITS & WINE,854,TAAKA,TAAKA VODKA 80,...,12,80.0,250.0,14580.00,332.50,58.320000,43.849624,0.1,3000.0,1
4,2016-12-01,1000325,SPECS,300001740,COWTOWN DISC LIQUOR SPF LTD,SPIRITS,BROWN FORMAN,5,JACK DANIELS TENNESSEE WHISKEY,JACK DANIELS BLK WHSKY,...,12,80.0,216.0,64748.16,287.28,299.760000,225.383459,0.1,2592.0,1


## Check nulls

In [17]:
cols=data.columns
data[cols].isna().sum()

Month                              0
Chain Master ID                    0
Chain Master                       0
Customer ID                        0
Customer                           0
Beverage Type                      0
Vendor                             0
Vendor ID                          0
Brand                              0
Label                              0
Size                               0
Product                            0
Product ID                         0
Fiscal Year                        0
Category (CatMan)                  0
Category Rank (CatMan)             0
Product Segment (CatMan)           0
Type (CatMan)                      0
General Flavor (CatMan)            0
Flavor (CatMan)                    0
Age Label (CatMan)             51058
Import or Domestic (CatMan)        0
Country (CatMan)                   0
Region (CatMan)                    0
Origin (CatMan)                    0
Premise                            0
Chain                              0
C

## Saving

In [18]:
path = os.path.join(folder,'tidy_data/Transactions_v3/')
print(path)
data.to_csv(os.path.join(path , "Transactions.csv"))
data.to_pickle(os.path.join(path,"Transactions.pkl"))

C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\tidy_data/Transactions_v3/


# ETL Pricing

## Reading original data

In [19]:
path = os.path.join(folder,'original_untidy/Pricing_v3/')
files =  glob.glob(path + "*.xlsx")
pricingRaw = pd.DataFrame()
for i,f in enumerate(files):
    print(f)
    pricingRaw = pricingRaw.append(pd.read_excel(f,skiprows=5),ignore_index=True)


pricingRaw.head()


C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\original_untidy/Pricing_v3\Pricing MSTR Data Spirits 6.12.20.xlsx


,Brand,Pricing Group,Deal (w / State No),Deal - Premise,Type,Qty,ASW Product #,Product,Size Descriptor,PK,Start,End,List Cs,List Btl,Case Disc,Btl Disc,Net Case,Net Btl
0,BONNIE ROSE,DELETED ITEMS,0,Both,LIST,1,1166396,BONNIE ROSE ORNG WHSKY 750M,750M,12,2009-02-01,2039-12-31,169.92,14.16,0.0,0.0,169.92,14.16
1,BONNIE ROSE,DELETED ITEMS,0,Both,LIST,1,1167396,BONNIE ROSE SPCD APL WHSKY 750M,750M,12,2009-02-01,2039-12-31,169.92,14.16,0.0,0.0,169.92,14.16
2,DUKE KENTUCKY,DUKE KY BBN 750M 12PK,0,Both,LIST,1,25150397,DUKE KENTUCKY STRAIGHT BBN 750M,750M,12,2009-02-01,2039-12-31,342.84,28.57,0.0,0.0,342.84,28.57
3,HOTEL TANGO,HOTEL TANGO BRAVO BBN 750M 6PK,0,Both,LIST,1,70182396,HOTEL TANGO BRAVO BBN 6PK 750M,750M,6,2009-02-01,2039-12-31,243.18,40.53,0.0,0.0,243.18,40.53
4,HOTEL TANGO,HOTEL TANGO BRAVO BBN 750M 6PK,0,Both,LIST,1,70182397,HOTEL TANGO BRAVO BBN 6PK PRIDE WRAP 750M,750M,6,2009-02-01,2039-12-31,243.18,40.53,0.0,0.0,243.18,40.53


## Transformation

In [20]:
# merge iwht STD Cases Table
# https://www.ttb.gov/distilled-spirits/conversion-tables 
cases = pd.DataFrame({"Size":['750M', '1.75L', '100M', '375M', '1L','200M', '50M','12Z','355M','5Z','UNIT']
                      ,"Size Liters":[.750,1.75 ,.1 ,.375 ,.1 ,.2 ,.05,0.355,0.355,0.148,0]
                      #,"Case Bottles":[12  ,6    ,60 ,24   ,12 ,48 ,120]
                      #,"Case Liters": [9   ,10.50,6  ,9    ,9  ,9.6,6]
                     })

pricing = pricingRaw.copy()
print(pricingRaw.shape)
pricing = pricing.drop(pricing[pricing['Size Descriptor']=='UNIT'].index)
print(pricing.shape)

# convert units to 9L cases equivalent
pricing=pd.merge(pricing,cases,how='left',left_on="Size Descriptor",right_on="Size")

#calculate totals
# https://www.inchcalculator.com/convert/case-beer-to-liter-beer/
pricing['List 9L'] = pricing['List Btl'] / pricing['Size Liters'] * 9.0
pricing['9L Disc'] = pricing['Btl Disc'] / pricing['Size Liters'] * 9.0
pricing['Net 9L'] = pricing['Net Btl'] / pricing['Size Liters'] * 9.0

#Removing size = UNIT as it doesn't liters


#data['Bottles']=data['Qty Per Case'] * data['STD. Cases']
#data['Liters']=data['Size Liters'] *  data['Bottles'] # conversion is more complex see above link, we need to use the 9l cases unit 
#data['Dollar Sales per Liter']=data['Dollar Sales per 9L Case'] * 9.0  # conversion is more complex see above link, we need to use the 9l cases unit

#adding a 'transactions' column
#data['Transactions']=1

#Removing nulls in dollar sales (it happens whnere STD Cases is 0 and there is a credit in Dollar Sales)
#data.dropna(subset=['STD. Cases','Dollar Sales per Case','Dollar Sales per 9L Case'],inplace=True)

## Remove negatives!
#print(data.shape)
#cols=['STD. Cases','Dollar Sales','9L Cases','Dollar Sales per Case','Dollar Sales per 9L Case']
#for c in cols: data=data.loc[data[c]>0]
#print(data.shape)
#print('check for negtives',[sum(data[c]<=0) for c in cols])

pricing.head()


(14361, 18)
(14344, 18)


,Brand,Pricing Group,Deal (w / State No),Deal - Premise,Type,Qty,ASW Product #,Product,Size Descriptor,PK,...,List Btl,Case Disc,Btl Disc,Net Case,Net Btl,Size,Size Liters,List 9L,9L Disc,Net 9L
0,BONNIE ROSE,DELETED ITEMS,0,Both,LIST,1,1166396,BONNIE ROSE ORNG WHSKY 750M,750M,12,...,14.16,0.0,0.0,169.92,14.16,750M,0.75,169.92,0.0,169.92
1,BONNIE ROSE,DELETED ITEMS,0,Both,LIST,1,1167396,BONNIE ROSE SPCD APL WHSKY 750M,750M,12,...,14.16,0.0,0.0,169.92,14.16,750M,0.75,169.92,0.0,169.92
2,DUKE KENTUCKY,DUKE KY BBN 750M 12PK,0,Both,LIST,1,25150397,DUKE KENTUCKY STRAIGHT BBN 750M,750M,12,...,28.57,0.0,0.0,342.84,28.57,750M,0.75,342.84,0.0,342.84
3,HOTEL TANGO,HOTEL TANGO BRAVO BBN 750M 6PK,0,Both,LIST,1,70182396,HOTEL TANGO BRAVO BBN 6PK 750M,750M,6,...,40.53,0.0,0.0,243.18,40.53,750M,0.75,486.36,0.0,486.36
4,HOTEL TANGO,HOTEL TANGO BRAVO BBN 750M 6PK,0,Both,LIST,1,70182397,HOTEL TANGO BRAVO BBN 6PK PRIDE WRAP 750M,750M,6,...,40.53,0.0,0.0,243.18,40.53,750M,0.75,486.36,0.0,486.36


## Check nulls

In [21]:
cols=pricing.columns
pricing[cols].isna().sum()

Brand                  0
Pricing Group          0
Deal (w / State No)    0
Deal - Premise         0
Type                   0
Qty                    0
ASW Product #          0
Product                0
Size Descriptor        0
PK                     0
Start                  0
End                    0
List Cs                0
List Btl               0
Case Disc              0
Btl Disc               0
Net Case               0
Net Btl                0
Size                   0
Size Liters            0
List 9L                0
9L Disc                0
Net 9L                 0
dtype: int64

## Saving

In [23]:
path = os.path.join(folder,'tidy_data/Pricing_v3/')
print(path)
pricing.to_csv(os.path.join(path , "Pricing.csv"))
pricing.to_pickle(os.path.join(path,"Pricing.pkl"))

C:\max\Google Drive Max\Max Shared\Master Data Science\2020-05 Capstone A\msds_capstone\data\tidy_data/Pricing_v3/
